In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def get_data():
    data = pd.read_csv("../raw_data/raw_data_v2.1.csv")
    return data

In [4]:
data = get_data()

/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_4530/1997837778.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../raw_data/raw_data_v2.1.csv")


In [5]:
def remove_221_rows(data):
    data = data[221:]
    return data

In [6]:
data = remove_221_rows(data)

In [7]:
data.shape

(118354, 115)

In [8]:
## Dropping rows if we have no betting data, and imputing if we have some betting data
def dropping_no_betting_data(data):
    data = data.dropna(subset=['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m' , 'f_pm_03m', 'f_pm_02m', 'f_pm_01m'], how='all')
    def impute_row(row):
        row = row.fillna(method='ffill').fillna(method='bfill')
        return row
    columns_to_impute = ['f_bsp', 'f_pm_15m', 'f_pm_10m', 'f_pm_05m', 'f_pm_03m', 'f_pm_02m', 'f_pm_01m']
    data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)
    return data

In [9]:
data = dropping_no_betting_data(data)

/var/folders/w6/wdkqbmf17b90n_r35p0cb7x00000gn/T/ipykernel_4530/2372925512.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_impute] = data[columns_to_impute].apply(impute_row, axis=1)


In [10]:
data.shape

(118093, 115)

In [11]:
def josh_features(data):

    #Creating country feature
    irish_tracks = [
    "SLIGO", "LIMERICK", "NAVAN", "WEXFORD", "CURRAGH",
    "GALWAY", "KILBEGGAN", "GOWRAN PARK", "BELLEWSTOWN",
    "LISTOWEL", "THURLES", "BALLINROBE", "TRAMORE",
    "LEOPARDSTOWN", "DOWN ROYAL", "ROSCOMMON", "CORK",
    "DUNDALK", "KILLARNEY", "LAYTOWN", "TIPPERARY",
    "FAIRYHOUSE", "NAAS", "DOWNPATRICK", "CLONMEL",
    "PUNCHESTOWN"
]

    data['country'] = data['f_track'].apply(lambda x: 'IRE' if x in irish_tracks else 'GB')

    #Completing f_class for Irish races

    # Calculate mean ratings for each 'f_id' group
    mean_ratings_by_id = data.groupby('f_id')['f_rating_or'].mean()

    # Define the mapping of mean ratings to f_class values
    rating_to_f_class_mapping = {
        (96, float('inf')): 1,
        (86, 96): 2,
        (76, 86): 3,
        (66, 76): 4,
        (56, 66): 5,
        (46, 56): 6,
        (-float('inf'), 46): 7
    }

    # Function to map mean ratings to f_class values
    def map_rating_to_f_class(mean_rating):
        for rating_range, f_class_value in rating_to_f_class_mapping.items():
            if rating_range[0] <= mean_rating <= rating_range[1]:
                return f_class_value

    # Apply the mapping to fill NULL values in 'f_class' column based on mean ratings
    data['f_class'] = data.apply(lambda row: map_rating_to_f_class(mean_ratings_by_id.get(row['f_id'])), axis=1)

    # Now the 'f_class' column should be filled based on the specified mapping using mean ratings

    # Merge the mean ratings back into the original DataFrame based on 'f_id'
    data = data.merge(mean_ratings_by_id, how='left', left_on='f_id', right_index=True)

    # Rename the merged mean rating column for clarity
    data.rename(columns={'f_rating_or_y': 'mean_f_rating_or_race', 'f_rating_or_x' : 'f_rating_or' }, inplace=True)

    # Create official rating vs average rating in the race feature
    data['or_rating_vs_avg_race'] = data['f_rating_or'] - data['mean_f_rating_or_race']

    # Create odds percentage and movement features
    data['15m_odds_prob'] = 1 / data['f_pm_15m']
    data['5m_odds_prob'] = 1 / data['f_pm_05m']
    data['15to5m_odds_move_perc'] = (data['5m_odds_prob'] / data['15m_odds_prob'] - 1)
    data['15to5m_odds_move_raw'] = (data['5m_odds_prob'] - data['15m_odds_prob'])

    return data

In [12]:
data = josh_features(data)

In [13]:
data['country'].value_counts()

country
GB     104206
IRE     13887
Name: count, dtype: int64

In [14]:
data.shape

(118093, 122)

In [15]:
def class_or_rating_average(data):
    average_or_rating_class = data.groupby("f_class")['f_rating_or'].mean().reset_index()
    average_or_rating_class.rename(columns={'f_rating_or': 'average_or_rating_class'}, inplace=True)
    data = data.merge(average_or_rating_class, on='f_class')
    data['above_below_official_rating_class'] = data['f_rating_or']- data['average_or_rating_class']
    return data

In [16]:
data = class_or_rating_average(data)

In [17]:
data.shape

(118093, 124)

In [18]:
def oli_features(data):
    data = data.sort_values(by='f_ko')
    data['PreviousPosition'] = data.groupby('f_horse')['f_place'].shift(fill_value=0)

    data = data.sort_values(by=['f_id', 'pred_isp'])
    data['PredictedRank'] = data.groupby('f_id').cumcount() + 1

    return data

In [19]:
data = oli_features(data)

In [20]:
data.shape

(118093, 126)

In [21]:
data.sample(50)

,id,f_id,f_ko,f_track,f_going,f_racetype,f_horse,f_jockey,f_trainer,f_distance,f_class,f_age,f_pace,f_weight,f_runners,pred_isp,f_rating_rbd,f_rating_or,f_stall,f_rating_hc,f_dob,f_prb_avg,f_lto_pos,f_headgear,f_place,f_bsp,f_pm_15m,f_pm_10m,f_pm_05m,f_pm_03m,f_pm_02m,f_pm_01m,f_bsp_p_back,f_bsp_p_lay,f_pm_01m_p_back,f_pm_01m_p_lay,f_pm_15m_p_back,f_pm_15m_p_lay,trainer_runs_win_at,trainer_runs_win_l200r,trainer_runs_win_l50r,trainer_runs_win_l16r,trainer_runs_at,trainer_runs_l200r,trainer_runs_l50r,trainer_runs_l16r,jockey_runs_win_at,jockey_runs_win_l200r,jockey_runs_win_l50r,jockey_runs_win_l16r,jockey_runs_at,jockey_runs_l200r,jockey_runs_l50r,jockey_runs_l16r,general_runs_win_at,general_runs_win_l200r,general_runs_win_l50r,general_runs_win_l16r,general_runs_at,general_runs_l200r,general_runs_l50r,general_runs_l16r,horse_runs_win_at,horse_runs_win_l10r,horse_runs_win_l5r,horse_runs_win_l2r,horse_runs_at,horse_runs_l10r,horse_runs_l5r,horse_runs_l2r,sum_bsp_trainer_at,sum_bsp_jockey_at,sum_bsp_horse_at,sum_bsp_trainer_l16r,sum_bsp_jockey_l16r,sum_bsp_trainer_l50r,sum_bsp_jockey_l50r,sum_bsp_trainer_l200r,sum_bsp_jockey_l200r,sum_bsp_horse_l10r,sum_bsp_horse_l5r,sum_bsp_horse_l2r,iv_horse_at,iv_trainer_l200r,iv_trainer_l50r,iv_trainer_l16r,iv_trainer_at,iv_jockey_l200r,iv_jockey_l50r,iv_jockey_l16r,iv_jockey_at,ae_horse_l10r,ae_horse_l5r,ae_horse_l2r,ae_horse_at,ae_trainer_l200r,ae_trainer_l50r,ae_trainer_l16r,ae_trainer_at,ae_jockey_l200r,ae_jockey_l50r,ae_jockey_l16r,ae_jockey_at,rolling_avg_trainer_finish_at,rolling_avg_trainer_finish_l200r,rolling_avg_trainer_finish_l50r,rolling_avg_trainer_finish_l16r,rolling_avg_horse_finish_at,rolling_avg_horse_finish_l10r,rolling_avg_horse_finish_l5r,rolling_avg_horse_finish_l2r,rolling_avg_jockey_finish_at,rolling_avg_jockey_finish_l200r,rolling_avg_jockey_finish_l50r,rolling_avg_jockey_finish_l16r,country,mean_f_rating_or_race,or_rating_vs_avg_race,15m_odds_prob,5m_odds_prob,15to5m_odds_move_perc,15to5m_odds_move_raw,average_or_rating_class,above_below_official_rating_class,PreviousPosition,PredictedRank
112054,16827789000249,16827789000027,2023-04-29 15:35:00,RIPON,SFT,Other Handicap,Its Good To Laugh,Hollie Doyle,Jennie Candlish,16.0,3,6,5.0,128,7,6.00,119.0,85,7.0,0-95,0.30,0.55,0.0,C,5,5.48,7.60,7.60,6.60,6.20,5.80,6.00,-1.00,0.95,-1.00,0.95,-1.00,0.95,7.0,7.0,3.0,2.0,100.0,100.0,50.0,16.0,200.0,28.0,8.0,3.0,1278.0,200.0,50.0,16.0,10072.0,21.0,3.0,2.0,97863.0,200.0,50.0,16.0,1.0,1.0,0.0,0.0,11.0,10.0,5.0,2.0,9.4466,184.4244,1.3463,0.6724,2.2550,3.8832,6.7501,9.4466,27.7759,1.2773,0.7966,0.5840,0.88,0.68,0.58,1.21,0.68,1.36,1.55,1.82,1.52,0.78,0.00,0.00,0.74,0.74,0.77,2.97,0.74,1.01,1.19,1.33,1.08,5.8200,5.8200,6.2800,6.2500,5.9091,5.6000,6.2000,3.5,4.8820,5.0250,4.8200,4.7500,GB,83.142857,1.857143,0.131579,0.151515,0.151515,0.019936,80.459746,4.540254,3,3
102845,16456311000015,16456311000009,2022-02-23 15:45:00,NEWCASTLE,STD,Other Handicap,Astro Jakk,Clifford Lee,K R Burke,7.0,3,6,4.0,131,5,10.00,118.0,89,1.0,0-95,0.38,0.39,5.0,NaN,3,25.00,25.00,29.00,30.00,26.00,25.00,23.00,-1.00,0.95,-1.00,0.95,-1.00,0.95,38.0,25.0,7.0,0.0,337.0,200.0,50.0,16.0,24.0,22.0,6.0,1.0,214.0,200.0,50.0,16.0,4715.0,23.0,6.0,3.0,45562.0,200.0,50.0,16.0,1.0,1.0,1.0,0.0,8.0,8.0,5.0,2.0,37.2753,22.4447,0.4519,2.2379,1.7522,6.6330,5.8006,22.6729,20.4936,0.4519,0.3287,0.2094,1.21,1.21,1.36,0.00,1.09,1.07,1.16,0.61,1.08,2.21,3.04,0.00,2.21,1.10,1.06,0.00,1.02,1.07,1.03,0.57,1.07,5.0804,4.9050,4.2600,4.1875,7.0000,7.0000,5.2000,6.0,5.5000,5.5650,5.1800,4.5000,GB,83.400000,5.600000,0.040000,0.033333,-0.166667,-0.006667,80.459746,8.540254,5,4
4171,16432935000307,16432935000001,2022-01-27 14:25:00,SOUTHWELL,STSL,Other Handicap,Zarzyni,Connor Beasley,David Barron,6.0,2,5,6.0,134,10,3.50,136.0,95,6.0,0-105,0.47,0.43,2.0,NaN,2,3.40,2.90,2.90,3.40,3.25,3.20,3.35,-1.00,0.95,-1.00,0.95,-1.00,0.95,30.0,30.0,8.0,4.0,213.0,200.0,50.0,16.0,57.0,24.0,7.0,1.0,392.0,200.0,50.0,16.0,4513.0,22.0,5.0,1.0,43850.0,200.0,50.0,16.

In [22]:
pd.set_option('display.max_rows', None)
data.isnull().sum()

id                                       0
f_id                                     0
f_ko                                     0
f_track                                  0
f_going                                  6
f_racetype                               0
f_horse                                  0
f_jockey                                 3
f_trainer                                1
f_distance                               0
f_class                                  0
f_age                                    0
f_pace                                  10
f_weight                                 0
f_runners                                0
pred_isp                                 0
f_rating_rbd                           251
f_rating_or                              0
f_stall                                609
f_rating_hc                          22340
f_dob                                13125
f_prb_avg                            18024
f_lto_pos                            18018
f_headgear 

In [23]:
data.to_csv("FE_v2_data.csv")

In [24]:
!pwd

/Users/joshstone/code/lucasglanville/and_theyre_off_backend/notebooks
